### Weekly Timeline Alignment for 4 OutcomeTables 
#### This notebook adds addtional feature (number of week from starting date) to cleaned version of outcome tables 

In [2]:
import pandas as pd
import numpy as np
import pandasql as ps
import time 
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [86]:
#import cleaned tables 
df_food = pd.read_csv('Result/Serving_clean.csv') 
df_sed = pd.read_csv('Result/sed_clean.csv') 
df_pa = pd.read_csv('Result/pa_clean.csv') 
df_pa_device = pd.read_csv('Result/pa_device_clean.csv') 
df_mvpa =  pd.read_csv('Result/pa_merged.csv') 
df_user = pd.read_csv('Raw Data/users.csv') 

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Starting Time Alignment (self-reported PA)

In [28]:
#all participants
id_list = []
for i in range(df_user.shape[0]):
    #extract users' name starting with 'mbc2'
    if (df_user['username'][i][0:4] == 'mbc2'):
        id_list.append(df_user['user_id'][i])
id_list.sort()
print('There are', len(id_list), 'participants in intervention/follow-up period')

There are 212 participants in intervention/follow-up period


In [29]:
df_pa= df_pa[df_pa['user_id'].isin(id_list)]

In [30]:
df_pa.head(5)

,user_id,upload_time,pa_minute_app
0,1,2012-09-06,20
1,1,2012-09-07,20
2,1,2012-09-10,20
3,1,2012-09-12,20
4,1,2012-09-16,20


In [31]:
#merge start_date to exsiting tables
#df_pa
df_pa = ps.sqldf('select d1.user_id, d1.upload_time, d1.pa_minute_app as upload_min, d2.start_date from df_pa as d1 left join df_user as d2 on d1.user_id = d2.user_id')
df_pa.head(5)

,user_id,upload_time,upload_min,start_date
0,1,2012-09-06,20,2012-08-27 00:00:00
1,1,2012-09-07,20,2012-08-27 00:00:00
2,1,2012-09-10,20,2012-08-27 00:00:00
3,1,2012-09-12,20,2012-08-27 00:00:00
4,1,2012-09-16,20,2012-08-27 00:00:00


In [37]:
#convert date to datetime type
df_pa['time1'] = df_pa['upload_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_pa['time2'] = df_pa['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [38]:
#get the time difference (in days)
df_pa['time_diff'] = df_pa['time1'] - df_pa['time2']

In [39]:
#visualize
df_pa['time_diff'] = df_pa['time_diff'].apply(lambda x: x.days)
df_pa.head(5)

,user_id,upload_time,upload_min,start_date,time1,time2,time_diff
0,1,2012-09-06,20,2012-08-27 00:00:00,2012-09-06,2012-08-27,10
1,1,2012-09-07,20,2012-08-27 00:00:00,2012-09-07,2012-08-27,11
2,1,2012-09-10,20,2012-08-27 00:00:00,2012-09-10,2012-08-27,14
3,1,2012-09-12,20,2012-08-27 00:00:00,2012-09-12,2012-08-27,16
4,1,2012-09-16,20,2012-08-27 00:00:00,2012-09-16,2012-08-27,20


In [40]:
#calculate the correponding weeks (around up)
df_pa['upload_week'] = df_pa['time_diff'].apply(lambda x: math.ceil(x/7))

In [41]:
#get columns of interest
pa_included = ['user_id', 'upload_time', 'upload_min', 'upload_week']
df_pa_align = df_pa[pa_included]

In [42]:
#visualize the results (self-reported MVPA)
df_pa_align.head(5)

,user_id,upload_time,upload_min,upload_week
0,1,2012-09-06,20,2
1,1,2012-09-07,20,2
2,1,2012-09-10,20,2
3,1,2012-09-12,20,3
4,1,2012-09-16,20,3


In [93]:
#Save as csv
df_pa_align.to_csv('Result_Weekly/pa_align.csv', index=False)

### Starting Time Alignment (Shimmer PA)

In [50]:
df_pa_device.head(5)

,user_id,upload_time,pa_minute_shimmer
0,1,2012-08-27,2
1,1,2012-09-07,2
2,1,2012-09-08,19
3,1,2012-09-11,5
4,1,2012-09-16,4


In [51]:
#merge start_date to exsiting tables
df_pa_device = ps.sqldf('select d1.user_id, d1.upload_time, d1.pa_minute_shimmer as upload_min, d2.start_date from df_pa_device as d1 left join df_user as d2 on d1.user_id = d2.user_id')
df_pa_device.head(5)

,user_id,upload_time,upload_min,start_date
0,1,2012-08-27,2,2012-08-27 00:00:00
1,1,2012-09-07,2,2012-08-27 00:00:00
2,1,2012-09-08,19,2012-08-27 00:00:00
3,1,2012-09-11,5,2012-08-27 00:00:00
4,1,2012-09-16,4,2012-08-27 00:00:00


In [52]:
#convert date to datetime type
df_pa_device['time1'] = df_pa_device['upload_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_pa_device['time2'] = df_pa_device['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#get the time difference (in days)
df_pa_device['time_diff'] = df_pa_device['time1'] - df_pa_device['time2']

#visualize
df_pa_device['time_diff'] = df_pa_device['time_diff'].apply(lambda x: x.days)
df_pa_device.head(5)

,user_id,upload_time,upload_min,start_date,time1,time2,time_diff
0,1,2012-08-27,2,2012-08-27 00:00:00,2012-08-27,2012-08-27,0
1,1,2012-09-07,2,2012-08-27 00:00:00,2012-09-07,2012-08-27,11
2,1,2012-09-08,19,2012-08-27 00:00:00,2012-09-08,2012-08-27,12
3,1,2012-09-11,5,2012-08-27 00:00:00,2012-09-11,2012-08-27,15
4,1,2012-09-16,4,2012-08-27 00:00:00,2012-09-16,2012-08-27,20


In [53]:
#calculate the correponding weeks (around up)
df_pa_device['upload_week'] = df_pa_device['time_diff'].apply(lambda x: math.ceil(x/7))

#get columns of interest
pa_included = ['user_id', 'upload_time', 'upload_min', 'upload_week']
df_pa_device_align = df_pa_device[pa_included]

#visualize the results (self-reported MVPA)
df_pa_device_align.head(5)

,user_id,upload_time,upload_min,upload_week
0,1,2012-08-27,2,0
1,1,2012-09-07,2,2
2,1,2012-09-08,19,2
3,1,2012-09-11,5,3
4,1,2012-09-16,4,3


In [59]:
#Save as csv
df_pa_device_align.to_csv('Result_Weekly/pa_device_align.csv', index=False)

### Starting Time Alignment (Merged PA)

In [78]:
df_mvpa= df_mvpa[df_mvpa['user_id'].isin(id_list)]
df_mvpa.head(5)

,user_id,upload_time,pa_minute_app,pa_minute_shimmer,MVPA_min
0,1,2012-09-06,20.0,0.0,20.0
1,1,2012-09-07,20.0,2.0,22.0
2,1,2012-09-10,20.0,0.0,20.0
3,1,2012-09-12,20.0,0.0,20.0
4,1,2012-09-16,20.0,4.0,24.0


In [80]:
#merge start_date to exsiting tables
df_mvpa = ps.sqldf('select d1.user_id, d1.upload_time, d1.MVPA_min as upload_min, d2.start_date from df_mvpa as d1 left join df_user as d2 on d1.user_id = d2.user_id')
df_mvpa.head(5)

,user_id,upload_time,upload_min,start_date
0,1,2012-09-06,20.0,2012-08-27 00:00:00
1,1,2012-09-07,22.0,2012-08-27 00:00:00
2,1,2012-09-10,20.0,2012-08-27 00:00:00
3,1,2012-09-12,20.0,2012-08-27 00:00:00
4,1,2012-09-16,24.0,2012-08-27 00:00:00


In [81]:
#convert date to datetime type
df_mvpa['time1'] = df_mvpa['upload_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_mvpa['time2'] = df_mvpa['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#get the time difference (in days)
df_mvpa['time_diff'] = df_mvpa['time1'] - df_mvpa['time2']

#visualize
df_mvpa['time_diff'] = df_mvpa['time_diff'].apply(lambda x: x.days)
df_mvpa.head(5)

,user_id,upload_time,upload_min,start_date,time1,time2,time_diff
0,1,2012-09-06,20.0,2012-08-27 00:00:00,2012-09-06,2012-08-27,10
1,1,2012-09-07,22.0,2012-08-27 00:00:00,2012-09-07,2012-08-27,11
2,1,2012-09-10,20.0,2012-08-27 00:00:00,2012-09-10,2012-08-27,14
3,1,2012-09-12,20.0,2012-08-27 00:00:00,2012-09-12,2012-08-27,16
4,1,2012-09-16,24.0,2012-08-27 00:00:00,2012-09-16,2012-08-27,20


In [82]:
#calculate the correponding weeks (around up)
df_mvpa['upload_week'] = df_mvpa['time_diff'].apply(lambda x: math.ceil(x/7))

#get columns of interest
pa_included = ['user_id', 'upload_time', 'upload_min', 'upload_week']
df_mvpa_align = df_mvpa[pa_included]

#visualize the results (self-reported MVPA)
df_mvpa_align.head(5)

,user_id,upload_time,upload_min,upload_week
0,1,2012-09-06,20.0,2
1,1,2012-09-07,22.0,2
2,1,2012-09-10,20.0,2
3,1,2012-09-12,20.0,3
4,1,2012-09-16,24.0,3


In [83]:
#Save as csv
df_mvpa_align.to_csv('Result_Weekly/pa_merged_align.csv', index=False)

### Starting Time Alignment (Food)

In [87]:
df_food= df_food[df_food['user_id'].isin(id_list)]
df_food.head(5)

,Food_ID,Food_ID2,Food_ID3,credit,Serving_ID1,Serving_ID2,size,fv_credit,time_upload,user_id,...,calories,protein,total_fat,total_carbohydrate,sugars,fiber,calcium,sodium,saturated_fatty_acids,cholesterol
0,1001129,1001129.0,1001129,0.0,190677,190677,50.0,0.0,2012-08-13 15:53:24,14,...,155.00,12.580000,10.610000,1.1200,1.120000,0.00000,50.0000,124.0000,3.267000,424.0
1,1159175,1159175.0,1159175,2.25,175688,175688,85.0,1.5,2012-08-13 15:53:24,14,...,22.50,1.500000,0.000000,4.5000,3.000000,6.00000,30.0000,0.0000,0.000000,0.0
2,1063636,1063636.0,1063636,0.5,108974,108974,255.0,2.0,2012-08-13 15:53:24,14,...,61.20,0.337875,0.082875,16.5240,15.306375,1.21125,7.0125,1.9125,0.004463,0.0
3,1063878,1063878.0,1063878,4.0,218795,218795,223.0,4.0,2012-08-13 16:31:43,14,...,115.96,0.579800,0.379100,30.7963,23.169700,5.35200,13.3800,2.2300,0.062440,0.0
4,1073329,1073329.0,1073329,1.5,218833,218833,136.0,1.5,2012-08-14 08:04:37,9,...,121.04,1.482400,0.448800,31.0624,16.632800,3.53600,6.8000,1.3600,0.152320,0.0


In [88]:
#merge start_date to exsiting tables
df_food = ps.sqldf('select d1.user_id, d1.time_upload as upload_time, d1.fv_credit, d1.total_fat, d2.start_date from df_food as d1 left join df_user as d2 on d1.user_id = d2.user_id')
df_food.head(5)

,user_id,upload_time,fv_credit,total_fat,start_date
0,14,2012-08-13 15:53:24,0.0,10.610000,2012-08-13 00:00:00
1,14,2012-08-13 15:53:24,1.5,0.000000,2012-08-13 00:00:00
2,14,2012-08-13 15:53:24,2.0,0.082875,2012-08-13 00:00:00
3,14,2012-08-13 16:31:43,4.0,0.379100,2012-08-13 00:00:00
4,9,2012-08-14 08:04:37,1.5,0.448800,2012-08-13 00:00:00


In [89]:
#convert date to datetime type
df_food['time1'] = df_food['upload_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df_food['time2'] = df_food['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#get the time difference (in days)
df_food['time_diff'] = df_food['time1'] - df_food['time2']

#visualize
df_food['time_diff'] = df_food['time_diff'].apply(lambda x: x.days)
df_food.head(5)

,user_id,upload_time,fv_credit,total_fat,start_date,time1,time2,time_diff
0,14,2012-08-13 15:53:24,0.0,10.610000,2012-08-13 00:00:00,2012-08-13 15:53:24,2012-08-13,0
1,14,2012-08-13 15:53:24,1.5,0.000000,2012-08-13 00:00:00,2012-08-13 15:53:24,2012-08-13,0
2,14,2012-08-13 15:53:24,2.0,0.082875,2012-08-13 00:00:00,2012-08-13 15:53:24,2012-08-13,0
3,14,2012-08-13 16:31:43,4.0,0.379100,2012-08-13 00:00:00,2012-08-13 16:31:43,2012-08-13,0
4,9,2012-08-14 08:04:37,1.5,0.448800,2012-08-13 00:00:00,2012-08-14 08:04:37,2012-08-13,1


In [90]:
#calculate the correponding weeks (around up)
df_food['upload_week'] = df_food['time_diff'].apply(lambda x: math.ceil(x/7))

#get columns of interest
df_food_included = ['user_id', 'upload_time', 'fv_credit', 'total_fat', 'upload_week']
df_food_align = df_food[df_food_included]

#visualize the results (self-reported MVPA)
df_food_align.head(5)

,user_id,upload_time,fv_credit,total_fat,upload_week
0,14,2012-08-13 15:53:24,0.0,10.610000,0
1,14,2012-08-13 15:53:24,1.5,0.000000,0
2,14,2012-08-13 15:53:24,2.0,0.082875,0
3,14,2012-08-13 16:31:43,4.0,0.379100,0
4,9,2012-08-14 08:04:37,1.5,0.448800,1


In [104]:
#Save as csv
df_food_align.to_csv('Result_Weekly/food_align.csv', index=False)

### Starting Time Alignment (SED)

In [95]:
df_sed= df_sed[df_sed['user_id'].isin(id_list)]
df_sed.head(5)

,user_id,when_sed,dayp1_minute,dayp2_minute,dayp3_minute,dayp4_minute,sed_total
0,14,2012-08-13,40,0,0,60,100
1,14,2012-08-14,60,0,30,0,90
2,9,2012-08-14,30,0,0,90,120
3,14,2012-08-15,0,60,0,180,240
4,9,2012-08-15,45,0,0,120,165


In [96]:
#merge start_date to exsiting tables
df_sed = ps.sqldf('select d1.user_id, d1.when_sed as upload_time, d1.sed_total as sed_min, d2.start_date from df_sed as d1 left join df_user as d2 on d1.user_id = d2.user_id')
df_sed.head(5)

,user_id,upload_time,sed_min,start_date
0,14,2012-08-13,100,2012-08-13 00:00:00
1,14,2012-08-14,90,2012-08-13 00:00:00
2,9,2012-08-14,120,2012-08-13 00:00:00
3,14,2012-08-15,240,2012-08-13 00:00:00
4,9,2012-08-15,165,2012-08-13 00:00:00


In [97]:
#convert date to datetime type
df_sed['time1'] = df_sed['upload_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_sed['time2'] = df_sed['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#get the time difference (in days)
df_sed['time_diff'] = df_sed['time1'] - df_sed['time2']

#visualize
df_sed['time_diff'] = df_sed['time_diff'].apply(lambda x: x.days)
df_sed.head(5)

,user_id,upload_time,sed_min,start_date,time1,time2,time_diff
0,14,2012-08-13,100,2012-08-13 00:00:00,2012-08-13,2012-08-13,0
1,14,2012-08-14,90,2012-08-13 00:00:00,2012-08-14,2012-08-13,1
2,9,2012-08-14,120,2012-08-13 00:00:00,2012-08-14,2012-08-13,1
3,14,2012-08-15,240,2012-08-13 00:00:00,2012-08-15,2012-08-13,2
4,9,2012-08-15,165,2012-08-13 00:00:00,2012-08-15,2012-08-13,2


In [98]:
#calculate the correponding weeks (around up)
df_sed['upload_week'] = df_sed['time_diff'].apply(lambda x: math.ceil(x/7))

#get columns of interest
sed_included = ['user_id', 'upload_time', 'sed_min', 'upload_week']
df_sed_align = df_sed[sed_included]

#visualize the results (self-reported MVPA)
df_sed_align.head(5)

,user_id,upload_time,sed_min,upload_week
0,14,2012-08-13,100,0
1,14,2012-08-14,90,1
2,9,2012-08-14,120,1
3,14,2012-08-15,240,1
4,9,2012-08-15,165,1


In [103]:
#Save as csv
df_sed_align.to_csv('Result_Weekly/sed_align.csv', index=False)